In [1]:
# !!pip install guardrails-ai

In [2]:
# !!pip install sqlvalidator

## Imports above required

In [3]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
import os,openai
from langchain.llms import OpenAI
import pandas as pd
from langchain import PromptTemplate


In [4]:
open_ai_key = '*****************'
os.environ['OPENAI_API_KEY'] = open_ai_key
openai.api_key = open_ai_key

In [5]:

def get_completion(prompt, model="gpt-3.5-turbo",temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [6]:
prompt = PromptTemplate.from_template(template = '''Given a user query {query} now generate a sql query for it.
Give a JSON repsonse''')

In [7]:
prompt = prompt.format(query = 'Select the name of the employee who has the highest salary.')

In [8]:
result = get_completion(prompt=prompt,temperature=0.0)

print("Answer:",result,"\n")

Answer: SQL Query:

SELECT name
FROM employees
WHERE salary = (SELECT MAX(salary) FROM employees);

JSON Response:

{
  "name": "John Doe"
} 



## With Guardrails


### Defining Rail spec.

In [9]:
rail_str = """
<rail version="0.1">
<output>
<string description="Generate SQL for the given natural language instruction." format="bug-free-sql" name="generated_sql" on-fail-bug-free-sql="reask"></string>
</output>
<prompt>

Generate a valid SQL query for the following natural language instruction:

${nl_instruction}

${gr.complete_json_suffix}
</prompt>
</rail>
"""

In [10]:
from guardrails.validators import BugFreeSQL
from pydantic import BaseModel, Field

prompt = """

Generate a valid SQL query for the following natural language instruction:

${nl_instruction}

${gr.complete_json_suffix}
"""


## defining a pydantic basemodel
class ValidSql(BaseModel):
    generated_sql: str = Field(description="Generate SQL for the given natural language instruction.", validators=[BugFreeSQL(on_fail="reask")])

In [11]:
import guardrails as gd

from rich import print

In [12]:
guard = gd.Guard.from_rail_string(rail_str)


In [13]:
guard = gd.Guard.from_pydantic(output_class=ValidSql, prompt=prompt)

In [14]:
print(guard.base_prompt)

Generate a valid SQL query for the following natural language instruction:

${nl_instruction}


Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <string name="generated_sql" format="bug-free-sql" description="Generate SQL for the given natural language 
instruction."/>
</output>


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise. If you are unsure anywhere, enter `null`.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`

In [15]:
import openai

raw_llm_response, validated_response = guard(
    openai.Completion.create,
    prompt_params={
        "nl_instruction": "Select the name of the employee who has the highest salary."
    },
    engine="text-davinci-003",
    max_tokens=2048,
    temperature=0,
)


Async event loop found, but guard was invoked synchronously.For validator parallelization, please call `validate_async` instead.


In [16]:
print(validated_response)

{'generated_sql': 'SELECT name FROM employee ORDER BY salary DESC LIMIT 1'}

In [17]:
print(guard.state.most_recent_call.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Generate a valid SQL query for the following natural language instruction:                              │ │
    │ │                                                                                                         │ │
    │ │ Select the name of the employee who has the highest salary.                                             │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <string name="generated_sql" format="bug-free-sql" description="Generate SQL for the given natural  │ │
    │ │ language instruction."/>                                                                                │ │
    │ │ </output>                                                                                               │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the │ │
    │ │ `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding  │ │
    │ │ XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g.        │ │
    │ │ requests for lists, objects and specific types. Be correct and concise. If you are unsure anywhere,     │ │
    │ │ enter `null`.                                                                                           │ │
    │ │                                                                                                         │ │
    │ │ Here are examples of simple (XML, JSON) pairs that show the expected behavior:                          │ │
    │ │ - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`                     │ │
    │ │ - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO',     │ │
    │ │ etc.]}`                                                                                                 │ │
    │ │ - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index"          │ │
    │ │ format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`                        │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Json Output:                                                                                            │ │
    │ │                                            

## Guard Rail for Dataframe Agent

In [18]:
rail_spec = """
<rail version="0.1">
<output>
<object name="movie_info">
<string description="Movie's Name" name="movie_name"></string>
<float format="valid-range: 0 10" name="ratings"></float>

</object>
</output>
<prompt>

Given information about movies figure out movie ratings based on user query
${user_query}. Give out JSON repsonse

${gr.complete_json_suffix_v2}
</prompt>
</rail>
"""

In [19]:
from rich import print

from langchain.output_parsers import GuardrailsOutputParser

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

In [20]:
output_parser = GuardrailsOutputParser.from_rail_string(rail_spec)

In [21]:
print(output_parser.guard.prompt)

Given information about movies figure out movie ratings based on user query
${user_query}. Give out JSON repsonse


Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <object name="movie_info">
        <string description="Movie's Name" name="movie_name"/>
        <float format="valid-range: min=0 max=10" name="ratings"/>
    </object>
</output>


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`

In [22]:
prompt = PromptTemplate(
    template=output_parser.guard.prompt.escape(),
    input_variables=output_parser.guard.prompt.variable_names,
)

In [23]:
import pandas as pd

In [24]:
df = pd.read_csv('/home/sahibpreet/Downloads/archive/PopularMovies.csv')

In [25]:
df.head()

,Unnamed: 0,Movie_Names,Ratings,Released_Year,Movie_Link,Votes,Metascore,Userreviews,Criticreviews
0,0,The Shawshank Redemption,9.2,1994,https://www.imdb.com/title/tt0111161/?pf_rd_m=...,"<div class=""sc-bde20123-3 bjjENQ"">2.8M</div>",82,10.9K,173
1,1,The Godfather,9.2,1972,https://www.imdb.com/title/tt0068646/?pf_rd_m=...,"<div class=""sc-bde20123-3 bjjENQ"">1.9M</div>",100,5.4K,203
2,2,The Dark Knight,9.0,2008,https://www.imdb.com/title/tt0468569/?pf_rd_m=...,"<div class=""sc-bde20123-3 bjjENQ"">2.7M</div>",84,8.5K,448
3,3,The Godfather Part II,9.0,1974,https://www.imdb.com/title/tt0071562/?pf_rd_m=...,"<div class=""sc-bde20123-3 bjjENQ"">1.3M</div>",90,1.3K,136
4,4,12 Angry Men,9.0,1957,https://www.imdb.com/title/tt0050083/?pf_rd_m=...,"<div class=""sc-bde20123-3 bjjENQ"">815K</div>",97,2K,165


In [26]:
query = 'What is the rating for 12 Angry Men'

In [27]:
prompt = prompt.format_prompt(user_query=query).to_string()

In [28]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0,openai_api_key=open_ai_key),  df, verbose=True)

In [29]:
agent.run(prompt)



> Entering new  chain...
Thought: I need to extract the movie name and rating from the dataframe
Action: python_repl_ast
Action Input: df[df['Movie_Names'] == '12 Angry Men']['Ratings'].to_json()
Observation: {"4":9.0}
Thought: I now know the final answer
Final Answer: {"movie_name": "12 Angry Men", "ratings": 9.0}

> Finished chain.


'{"movie_name": "12 Angry Men", "ratings": 9.0}'